In [5]:
from fyers_apiv3 import fyersModel
from indicators import alligator, vroc, sma
import pandas as pd
import numpy as np
import time
client_id = "ZHQ4IJL7TI-100"
with open("access_token", "r") as f:
    access_token = f.read()

fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

data = {
    "symbol": "NSE:ZEEL-EQ",
    "resolution": "1",
    "date_format": "1",
    "range_from": "2024-07-01",
    "range_to": "2024-10-07",
    "cont_flag": "1"
}

response = fyers.history(data=data)
len(response['candles']), response

# with open("data/zeel/NSE:ZEEL-EQ", 'w') as file:
#     file.write("Open,High,Low,Close,Volume")
#
#     for i in response['candles']:
#         file.write(f"\n{i[1]},{i[2]},{i[3]},{i[4]},{i[5]}")



(25500,
 {'candles': [[1719805500, 152, 152.1, 151.2, 151.32, 237191],
   [1719805560, 151.32, 151.48, 151, 151.46, 70362],
   [1719805620, 151.44, 151.45, 151.3, 151.4, 37558],
   [1719805680, 151.4, 151.75, 151.4, 151.75, 27007],
   [1719805740, 151.8, 151.99, 151.7, 151.99, 70949],
   [1719805800, 151.99, 151.99, 151.62, 151.8, 41893],
   [1719805860, 151.75, 151.79, 151.7, 151.77, 21841],
   [1719805920, 151.77, 151.92, 151.74, 151.88, 43935],
   [1719805980, 151.85, 151.85, 151.79, 151.8, 31012],
   [1719806040, 151.79, 151.85, 151.74, 151.75, 59095],
   [1719806100, 151.74, 151.79, 151.7, 151.79, 5164],
   [1719806160, 151.79, 151.85, 151.61, 151.61, 32252],
   [1719806220, 151.61, 151.69, 151.41, 151.49, 35828],
   [1719806280, 151.5, 151.5, 151.26, 151.3, 64961],
   [1719806340, 151.3, 151.5, 151.27, 151.49, 63808],
   [1719806400, 151.49, 151.49, 151.4, 151.48, 8120],
   [1719806460, 151.44, 151.48, 151.29, 151.3, 22149],
   [1719806520, 151.3, 151.41, 151.25, 151.4, 39719],
 

In [2]:
temp = []
for c in response['candles']:
    temp.append(sum(c[2:4])/2)

alli = alligator(pd.Series(temp))
alli = alli.to_numpy()
alli

array([[         nan,          nan,          nan],
       [         nan,          nan,   0.        ],
       [         nan,          nan,   0.        ],
       ...,
       [126.34450037, 126.08371241, 125.9676008 ],
       [126.35069265, 126.03137336, 125.95908064],
       [126.35756244, 125.98370169, 125.98326451]])

In [15]:
dates = []
X = []
k = 0
for i in range(len(response['candles'])-1):
# for i in range(10):
    candle = response["candles"][i]
    d = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(candle[0]))
    if d not in dates:
        dates.append(d)
        temp = []
        temp.append(d)
        temp.append(candle[1])
        temp.append(candle[2])
        temp.append(candle[3])
        temp.append(candle[4])
        temp.append(round(alli[i][0], 2))
        temp.append(round(alli[i][1], 2))
        temp.append(round(alli[i][2], 2))
        temp.append(round(sma(np.array(response['candles'])[:i+1, 4], 5)[-1], 2))
        temp.append(round(sma(np.array(response['candles'])[:i+1, 4], 15)[-1], 2))
        temp.append(round(vroc(np.array(response['candles'])[:i, 5], 14), 2))
        X.append(temp)
        print("id:", i, "\tdata:", temp)
    else:
        k += 1
        print("skipping", k, "\t", d)

id: 0 	data: ['2024-07-01 09:15:00', 152, 152.1, 151.2, 151.32, nan, nan, nan, nan]
id: 1 	data: ['2024-07-01 09:16:00', 151.32, 151.48, 151, 151.46, nan, nan, 0.0, nan]
id: 2 	data: ['2024-07-01 09:17:00', 151.44, 151.45, 151.3, 151.4, nan, nan, 0.0, nan]
id: 3 	data: ['2024-07-01 09:18:00', 151.4, 151.75, 151.4, 151.75, nan, nan, 0.0, nan]
id: 4 	data: ['2024-07-01 09:19:00', 151.8, 151.99, 151.7, 151.99, nan, nan, 0.0, nan]
id: 5 	data: ['2024-07-01 09:20:00', 151.99, 151.99, 151.62, 151.8, nan, nan, 151.54, nan]
id: 6 	data: ['2024-07-01 09:21:00', 151.75, 151.79, 151.7, 151.77, nan, nan, 151.59, nan]
id: 7 	data: ['2024-07-01 09:22:00', 151.77, 151.92, 151.74, 151.88, nan, nan, 151.62, nan]
id: 8 	data: ['2024-07-01 09:23:00', 151.85, 151.85, 151.79, 151.8, nan, nan, 151.66, nan]
id: 9 	data: ['2024-07-01 09:24:00', 151.79, 151.85, 151.74, 151.75, nan, nan, 151.69, nan]


[['2024-07-01 09:15:00', 152, 152.1, 151.2, 151.32, nan, nan, nan, nan],
 ['2024-07-01 09:16:00', 151.32, 151.48, 151, 151.46, nan, nan, 0.0, nan],
 ['2024-07-01 09:17:00', 151.44, 151.45, 151.3, 151.4, nan, nan, 0.0, nan],
 ['2024-07-01 09:18:00', 151.4, 151.75, 151.4, 151.75, nan, nan, 0.0, nan],
 ['2024-07-01 09:19:00', 151.8, 151.99, 151.7, 151.99, nan, nan, 0.0, nan],
 ['2024-07-01 09:20:00', 151.99, 151.99, 151.62, 151.8, nan, nan, 151.54, nan],
 ['2024-07-01 09:21:00', 151.75, 151.79, 151.7, 151.77, nan, nan, 151.59, nan],
 ['2024-07-01 09:22:00',
  151.77,
  151.92,
  151.74,
  151.88,
  nan,
  nan,
  151.62,
  nan],
 ['2024-07-01 09:23:00', 151.85, 151.85, 151.79, 151.8, nan, nan, 151.66, nan],
 ['2024-07-01 09:24:00',
  151.79,
  151.85,
  151.74,
  151.75,
  nan,
  nan,
  151.69,
  nan]]

In [16]:
with open("abc.csv", 'w') as f:
    f.write("DATE," + "OPEN," + "HIGH," + "LOW," + "CLOSE," + "JAW," + "TEETHS," + "LIPS," + "VROC")
    for x in X:
        f.write("\n" + str(x)[1:-1])
            